In [13]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.utils
import torch.nn.init
from torch.utils.data import DataLoader

In [14]:
mnist_train = dsets.MNIST(root = 'E:\MNIST/', train = True, transform = transforms.ToTensor(), download = True)
mnist_test = dsets.MNIST(root = 'E:\MNIST/', train = False, transform = transforms.ToTensor(), download = True)

In [16]:
batch_size = 100
epochs = 10
data_loader = DataLoader(dataset = mnist_train, batch_size = batch_size)

In [17]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5
        # L1 ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L2 ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L3 ImgIn shape=(?, 7, 7, 64)
        #    Conv      ->(?, 7, 7, 128)
        #    Pool      ->(?, 4, 4, 128)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))

        # L4 FC 4x4x128 inputs -> 625 outputs
        self.fc1 = torch.nn.Linear(4 * 4 * 128, 625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1 - self.keep_prob))
        # L5 Final FC 625 inputs -> 10 outputs
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [18]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [19]:
model = CNN().to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [21]:
total_batch = len(data_loader)
# train
for epoch in range(epochs):
    avg_cost = 0

    for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        out = model(X)
        loss = criterion(out, Y)
        loss.backward()
        optimizer.step()

        avg_cost += loss / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.188206807
[Epoch:    2] cost = 0.0533717014
[Epoch:    3] cost = 0.0360256545
[Epoch:    4] cost = 0.0286221895
[Epoch:    5] cost = 0.0217051283
[Epoch:    6] cost = 0.018670762
[Epoch:    7] cost = 0.0163670238
[Epoch:    8] cost = 0.0129882628
[Epoch:    9] cost = 0.0126524568
[Epoch:   10] cost = 0.0129922973


In [38]:
# Test
import pandas as pd
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
#     X_test = mnist_test.test_data.to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())
    
    df = pd.DataFrame([accuracy.item()], columns=['Accuracy'])
    df.to_csv('E:\MNIST/MNIST_Acc.csv')

Accuracy: 0.9855999946594238


In [30]:
data = pd.read_csv('E:\MNIST/MNIST_Acc.csv')
type(data)

pandas.core.frame.DataFrame